In [15]:
!pip install googletrans

In [3]:
import os
import sys
import pandas as pd
import googletrans
from googletrans import Translator
%matplotlib inline
import matplotlib.pyplot as plt

In [24]:
DIR_PATH = r"C:\Users\gabi9\Desktop\Dev\python_dev\elections"
CALPI = "calpi"
SETTELMENT = "settelments"
translator = Translator()



def get_data(agg_type, num):
    """returns dataframe of the requested csv
    
    parameters
    ----------
    agg_type - aggregation type of the csv needed
    num - election number
    
    return
    ------
    DataFrame
    """
    if type(num) is not str:
        num = str(num)
    
    ROOT_PATH = os.path.abspath(os.getcwd())
    path = os.path.join(ROOT_PATH,num+agg_type+".xlsx")
    df = pd.read_excel(path)
    df_en = df.copy()
    
    df_en.rename(columns=lambda x: translator.translate(x,dest='en',src='he').text, inplace=True)
    return df_en
    

In [2]:
CALPI = "calpi"
SETTELMENT = "settelments"

In [8]:
def getdata(agg_type , num):
    
    if type(num) is not str:
        num = str(num)
    ROOT_PATH = os.path.abspath(os.getcwd())
    path = os.path.join(ROOT_PATH,num+agg_type+".xlsx")
    df = pd.read_excel(path)
    return df

def getRideofColm(df):
    dict = {}
    for column in df.columns[6:]:
       print(column)
       dict[column]=df[column].sum()
    for c in dict:
        if dict[c] < 10000:
           df = df.drop(c, 1)
    for i,r in df.iterrows():
        for c in df.columns[6:]:
            x = r[c]/r['valid votes']*100
            df.at[i , c]=x

    return df
        


    
df21=getdata(SETTELMENT,21)
df21=getRideofColm(df21)
print(df21.columns)

df22=getdata(SETTELMENT,22)
df22=getRideofColm(df22)
print(df22.columns)

df23=getdata(SETTELMENT,23)
df23=getRideofColm(df23)
print(df23.columns)

#plt.figure(1)
#plt.plot(df21['City name'], df21['Number of polling'], color='b')
#plt.xlabel('City name')
#plt.ylabel('Number of polling')
#plt.title('')

valid votes
Avoda
United Torah Judaism
UAL-Balad
Hadash
Zehut
Senior Citizens Party
an eternal covenant
Yahad
Zomet
Consolidation of right-wing parties
Yashar
Biblical Bloc Party
Union allies
Responsibility Founders
Kolano
Hope for change
Israel is our Home
Likud
Meretz
New Right
Our rights, our voice
Reforma
Protector
Gesher
Betah
Human Dignity
Social Leadership
Blue and white
Pirate
simple love
Education
Equal
We are all friends nach
social justice
Green economy
Ofek Hadash
Me and you
Justice for all
Our Land of Israel
at first
All Israeli brothers and action in Israel
Arab List
Shas
Index(['committee code', 'code', 'name', 'Holders of voting rights', 'Voters',
       'Disqualified', 'valid votes', 'Avoda', 'United Torah Judaism',
       'UAL-Balad', 'Hadash', 'Zehut', 'Consolidation of right-wing parties',
       'Kolano', 'Israel is our Home', 'Likud', 'Meretz', 'New Right',
       'Gesher', 'Blue and white', 'Shas'],
      dtype='object')
valid votes
Gesher Avoda
United Torah Juda

TypeError: unsupported operand type(s) for +: 'int' and 'str'